## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a> 2024/6/18

#### 这里学习的是2024年kddcup的Whoiswho赛道的top12solution.github链接如下:<a href="https://github.com/Leo1998-Lu/KDD2024-WhoIsWho">Leo1998-Lu</a>.这里重点学习的是作者的特征工程,所以只学习了作者的train.py文件.我删除了作者的无用代码,并修正了原作者一部分错误代码,给代码添加了一定的注释方便阅读,并写下了我所学到的新东西以及我的心得体会。


### 1.导入必要的库,并且固定随机种子,这里没什么好说的。

In [1]:
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
import json#用于读取和写入json数据格式
import re#用于正则表达式提取
#model lgb分类模型,日志评估,早停防止过拟合
from  lightgbm import LGBMClassifier,log_evaluation,early_stopping
#metric
from sklearn.metrics import roc_auc_score#导入roc_auc曲线
#KFold是直接分成k折,StratifiedKFold还要考虑每种类别的占比
from sklearn.model_selection import StratifiedGroupKFold
from tqdm import tqdm#加载进度条的库
from collections import Counter#用于对一组元素计数
from sklearn.preprocessing import LabelEncoder#将分类数据转换成数值格式
#两个词转换成向量的常用模型,CountVectorizer就是简单的词频统计,TfidfVectorizer则是tfidf模型
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#gensim提供了许多文本向量化的工具和算法
from gensim.models import Word2Vec#用来生成词嵌入的算法
#转小写,去除标点符号,分词.例如:'He is a very smart boy.'->['he', 'is', 'very', 'smart', 'boy']
from gensim.utils import simple_preprocess
import gc#垃圾回收模块
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

### 2.读取数据,这里也没什么好说的。

In [2]:
with open("/kaggle/input/2024kddcupwhoiswho/2024kddcupwhoiswho/train_author.json",encoding='utf-8') as f:
    train_author=json.load(f)
with open("/kaggle/input/2024kddcupwhoiswho/2024kddcupwhoiswho/ind_test_author_filter_public.json",encoding='utf-8') as f:
    test_author=json.load(f)
with open("/kaggle/input/2024kddcupwhoiswho/2024kddcupwhoiswho/pid_to_info_all.json",encoding='utf-8') as f:
    pid_to_info=json.load(f)

### 3.构造训练数据.

#### 这里的初始特征除了label有13个，下面是详细含义。

- author_id:就是一个编号,不具有具体的含义.

- name:作者名字.

- author:原始数据中的列表,里面有作者名和组织的字典。(例如[{'name':XXX,'org':XXX}])

- venue:就是把原始数据中地点稍微做了一些文本的处理(clean_text)

- keywords:原来是一个字符串的列表,例如['a','b'],结果变成字符串"['a','b']"后做了文本的处理。(这里比较巧妙的是如果keywords是空列表就用title来代替。)

- text: title+空格+keywords+空格+abstract

- x1~x6:title的长度、abstract的长度、keywords的个数、作者的个数、2024-year,地点的长度.这里不知道为什么要2024-year,传入year和2024-year对树模型应该是没有影响的。

- top_author:这里选择前5个作者用逗号拼接在了一起。

#### 我学到了什么:

##### 1.首先是对抗学习,这个任务中正负样本比例是不平衡的,负样本约为正样本的十分之一。他这里对负样本做了2个label,一次构造成正样本,一次构造成负样本,叫做对抗学习。据我目前的理解,样本同时为正样本和负样本,这样模型对于这些样本肯定学不好,如果预测为正样本,那么负样本的损失就会大,预测为负样本,那么正样本的损失就会大,模型就会更加努力的学习这些样本,从而就能学的比较好。也有可能是最后预测概率为0.5的时候损失会最小,正负样本比例不平衡的时候对负样本的预测概率为0.5也已经很不错了。

##### 2.keywords为空列表的时候用title来替代,这点我没有想到.title,abstract,keywords都含有文章的主要信息,abstract太长,长度和keywords相差太大,所以当keywords缺失的时候就可以用title来填充。

##### 3.top5author.有些论文可能会有几千个作者,如果把全部作者都进行统计很容易会有噪音.对论文越重要的作者往往排的位置会越靠前,所以他这里选择了top5作者来进行统计。

In [3]:
#对文本进行清理的函数
def clean_text(text):
    #删除标点符号,至于为什么删除这些也许是试出来效果好吧,要问就问原作者
    punctuation=['[',']','{','}',"'",':']
    for p in punctuation:
        text = text.replace(p,'')
    #字符串转小写
    text = text.lower()
    return text

df=[]
labels=[]
columns=['author_id',#就是一个编号
              'name',#作者的名字
              'author',#原始数据中['name','org']
              'venue',#原始数据地点(clean)
              'keywords',#原始数据中列表变成字符串,keywords没有用title替代.
              'text',#title+keywords+abstract
              'x1',#len_title
              'x2',#len_abstract
              'x3',#len_keywords
              'x4',#len_authors
              'x5',#2024-year
              'x6',#len(venue)
              'top_author',#前5个作者拼接
]
for id,person_info in train_author.items():
    names = person_info['name'].lower()#作者的名字
    for text_id in person_info['normal_data']:
        feat=pid_to_info[text_id]
        #一篇论文选取前5个作者
        author_names = [x['name'] for x in feat['authors'][:5]]
        #作者用逗号拼接在一起
        ans = ','.join(author_names)
        #text是title+keywords+abstract,中间用空格隔开。
        text = str(feat['title'])+' '+ clean_text(str(feat['keywords']))+' '+str(feat['abstract'])
        #feat['keywords']=[],则str(feat['keywords'])='[]',len()=2
        if len(str(feat['keywords']))<3:#如果没有关键词就用title来替换关键词
            keywords = feat['title'].lower()
        else:#为了把[]去掉就需要clean_text函数
            keywords = clean_text(str( feat['keywords'])) 
        try:
            df.append(
                [id,names,feat['authors'],clean_text(str(feat['venue'])),keywords,text,len(feat['title']),len(feat['abstract']),len(feat['keywords']),len(feat['authors'])
                 ,2024-int(feat['year']), len(feat['venue']),ans]
                 )
        except:
            df.append(
                [id,names,feat['authors'],clean_text(str(feat['venue'])),keywords,text,len(feat['title']),len(feat['abstract']),len(feat['keywords']),len(feat['authors'])
                 ,20,10,ans]
                 )
        labels.append(1)
    for text_id in person_info['outliers']:
        feat=pid_to_info[text_id]
        author_names = [x['name'] for x in feat['authors'][:5]]
        ans = ','.join(author_names)
        text = feat['title']+ clean_text(str(feat['keywords'])) +feat['abstract']
   
        if len(str(feat['keywords']))<3:
            keywords = feat['title'].lower()
        else:
            keywords = clean_text(str( feat['keywords']))
        #对抗性学习 一个样本既标记成1又标记为0 
        for i in range(2):
            try:
                df.append(
                    [id,names,feat['authors'],clean_text(str(feat['venue'])),keywords,text,
                     len(feat['title']),len(feat['abstract']),len(feat['keywords']),len(feat['authors'])
                     ,2024-int(feat['year']), len(feat['venue']),ans]
                     )
            except:
                df.append(
                    [id,names,feat['authors'],clean_text(str(feat['venue'])),keywords,text,
                     len(feat['title']),len(feat['abstract']),len(feat['keywords']),len(feat['authors'])
                     ,20,10,ans]
                     )
            labels.append(i)
labels=np.array(labels)
df=pd.DataFrame(df)
df['label']=labels
print(f"df.shape:{df.shape},labels.shape:{labels.shape}")
print(f"np.mean(labels):{np.mean(labels)}")
df.columns = columns+['label']
df.head()

df.shape:(165594, 14),labels.shape:(165594,)
np.mean(labels):0.895618198727007


,author_id,name,author,venue,keywords,text,x1,x2,x3,x4,x5,x6,top_author,label
0,Iki037dt,atsushi ochiai,"[{'name': 'Hideaki Bando', 'org': ''}, {'name'...",annals of oncology,the power of the high-sensitive kras test to p...,THE POWER OF THE HIGH-SENSITIVE KRAS TEST TO P...,120,0,0,14,14,18,"Hideaki Bando,Takayuki Yoshino,Kazuya Tuchihar...",1
1,Iki037dt,atsushi ochiai,"[{'name': 'jintetsu soh', 'org': ''}, {'name':...",the journal of urology,1812 nicardipine versus saline injection as tr...,1812 NICARDIPINE VERSUS SALINE INJECTION AS TR...,123,0,0,8,13,22,"jintetsu soh,yoshio naya,atsushi ochiai,yasuyu...",1
2,Iki037dt,atsushi ochiai,"[{'name': 'M Inaba', 'org': 'Department of Uro...",ultrasound in medicine & biology,"vesicoureteral reflux, bladder hypertrophy, ch...",Possible use of ultrasound estimated bladder w...,100,986,4,9,23,32,"M Inaba,O Ukimura,A Kawauchi,T Iwata,M Kanazawa",1
3,Iki037dt,atsushi ochiai,"[{'name': 'hisao ito', 'org': ''}, {'name': 'm...",journal of steroid biochemistry,interaction between epidermal growth factor an...,Interaction between epidermal growth factor an...,103,0,0,10,37,31,"hisao ito,masanobu yamamoto,naruhito oda,fumio...",1
4,Iki037dt,atsushi ochiai,"[{'name': 'yasutoshi kuboki', 'org': 'Departme...",annals of oncology,"gastric cancer, amplification, mutation, next-...",Comprehensive Analyses Using Next-Generation S...,133,1629,5,10,9,18,"yasutoshi kuboki,seigo yamashita,tohru niwa,to...",1


### 测试数据的特征构造和训练数据一致,这里就不做重复说明了。

In [4]:
test_feats=[]
for id,person_info in test_author.items():
    names = person_info['name'].lower()
    for text_id in person_info['papers']:
        feat=pid_to_info[text_id]
        author_names = [x['name'] for x in feat['authors'][:5]]
        ans = ','.join(author_names)
        text = str(feat['title'])+' '+ clean_text(str(feat['keywords']))+' '+str(feat['abstract'])

        if len(str(feat['keywords']))<3:
            keywords = feat['title'].lower()
        else:
            keywords = clean_text(str(feat['keywords']))
        try:
            test_feats.append(
                [id,names, feat['authors'],clean_text(str(feat['venue'])),keywords,text, len(feat['title']),len(feat['abstract']),len(feat['keywords']),len(feat['authors'])
                 ,2024-int(feat['year']), len(feat['venue']),ans]
                 )
        except:
            test_feats.append(
                [id,names, feat['authors'],clean_text(str(feat['venue'])),keywords,text, len(feat['title']),len(feat['abstract']),len(feat['keywords']),len(feat['authors'])
                 ,20,10,ans]
                 )
test_feats=pd.DataFrame(test_feats)
print(f"valid_feats.shape:{test_feats.shape}")
test_feats.columns = columns
test_feats.head()

valid_feats.shape:(116262, 13)


,author_id,name,author,venue,keywords,text,x1,x2,x3,x4,x5,x6,top_author
0,Fkb16wn7,jitendra malik,"[{'name': 'Saurabh Gupta', 'org': 'university ...",eccv (7),learning rich features from rgb-d images for o...,Learning Rich Features from RGB-D Images for O...,79,1295,0,4,10,8,"Saurabh Gupta,Ross B. Girshick,Pablo Arbelaez,..."
1,Fkb16wn7,jitendra malik,"[{'name': 'João Carreira', 'org': 'University ...",computer vision and pattern recognition,human pose estimation with iterative error fee...,Human Pose Estimation with Iterative Error Fee...,51,1017,0,4,8,39,"João Carreira,pulkit agrawal,katerina fragkiad..."
2,Fkb16wn7,jitendra malik,"[{'name': 'Ashish Kumar', 'org': 'Univ Calif B...",advances in neural information processing syst...,visual memory,Visual Memory for Robust Path Following. visua...,40,710,1,5,6,64,"Ashish Kumar,Saurabh Gupta,David F. Fouhey,Ser..."
3,Fkb16wn7,jitendra malik,"[{'name': 'Li Yi', 'org': ''}, {'name': 'Lin S...",arxiv computer vision and pattern recognition,large-scale 3d shape reconstruction and segmen...,Large-Scale 3D Shape Reconstruction and Segmen...,74,687,0,50,7,46,"Li Yi,Lin Shao,Manolis Savva,Haibin Huang,Yang..."
4,Fkb16wn7,jitendra malik,"[{'name': 'Ke Li', 'org': 'University of Calif...",icml,fast k-nearest neighbour search via prioritize...,Fast k-Nearest Neighbour Search via Prioritize...,52,1061,0,2,7,4,"Ke Li,Jitendra Malik"


### 4.gg函数

### 首先看这个作者是不是top5作者,这非常重要,如果一篇论文有1000个作者,这个作者又是最后一个,那可能就是挂个名字,没有做什么贡献。然后统计了字符串的长度特征,其中author是列表,keywords已经在前面的操作中转成列表了,text是文本,top_author也是文本.将文本转成小写这个操作很常规.最后Labelencoder对作者id和地点转成一个数值.

In [5]:
def gg(df):
    
    #一个作者是不是top5主要作者
    author_ex = [] 
    for x,y in zip(df['name'],df['top_author']):
        author_ex.append(int(x in y))
    df['author_ex'] = author_ex
    
    for x in ['author','keywords','text','top_author']:
        df[f'{x}_len'] = df[x].apply(lambda x: len(str(x)))
        
    for col in ['text','top_author','name']:#将文本转成小写很常规
        df[col] = df[col].apply(lambda x: x.lower())
    
    le = LabelEncoder()
    df['name'] = le.fit_transform(df['author_id'])
    df['venue'] = le.fit_transform(df['venue'])
    df['venue'] = df['venue'].apply(lambda x: str(x))

    return df
train = gg(df)
valid = gg(test_feats)
del df,test_feats
gc.collect()#手动触发垃圾回收,强制回收由垃圾回收器标记为未使用的内存
valid.head()

,author_id,name,author,venue,keywords,text,x1,x2,x3,x4,x5,x6,top_author,author_ex,author_len,keywords_len,text_len,top_author_len
0,Fkb16wn7,127,"[{'name': 'Saurabh Gupta', 'org': 'university ...",6079,learning rich features from rgb-d images for o...,learning rich features from rgb-d images for o...,79,1295,0,4,10,8,"saurabh gupta,ross b. girshick,pablo arbelaez,...",0,252,79,1376,60
1,Fkb16wn7,127,"[{'name': 'João Carreira', 'org': 'University ...",5249,human pose estimation with iterative error fee...,human pose estimation with iterative error fee...,51,1017,0,4,8,39,"joão carreira,pulkit agrawal,katerina fragkiad...",1,287,51,1070,64
2,Fkb16wn7,127,"[{'name': 'Ashish Kumar', 'org': 'Univ Calif B...",2261,visual memory,visual memory for robust path following. visua...,40,710,1,5,6,64,"ashish kumar,saurabh gupta,david f. fouhey,ser...",0,407,13,765,71
3,Fkb16wn7,127,"[{'name': 'Li Yi', 'org': ''}, {'name': 'Lin S...",3017,large-scale 3d shape reconstruction and segmen...,large-scale 3d shape reconstruction and segmen...,74,687,0,50,7,46,"li yi,lin shao,manolis savva,haibin huang,yang...",0,1847,74,763,51
4,Fkb16wn7,127,"[{'name': 'Ke Li', 'org': 'University of Calif...",8078,fast k-nearest neighbour search via prioritize...,fast k-nearest neighbour search via prioritize...,52,1061,0,2,7,4,"ke li,jitendra malik",0,127,52,1115,20


### 5.词向量特征

#### 这里用tf-idf模型对text,keywords,venue构造特征,用词频统计模型对text,keywords,venue,topauthor构造特征,用word2vec对text构造特征.

#### 我其实在比赛中也想过构造词向量特征,不过只统计过title和abstract的top100,同时我是面向训练数据建模的。它这里是面向全部数据建模,虽然不知道允不允许,但是训练集和测试集一致的话效果肯定会更好。

#### text里有title+keywords+abstract,信息是最多的,所以用3个模型对其建模,topauthor信息是最少的,所以就用一个模型给它建模.

In [6]:
#将训练数据和测试数据拼接
df = pd.concat([train,valid],axis=0).reset_index(drop=True)
del train,valid
gc.collect()#手动触发垃圾回收,强制回收由垃圾回收器标记为未使用的内存
df['id'] = df.index
#用tfidf将text转成词向量的形式,具体参数没研究过.
vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(1,4),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True,max_features=100
)
X = vectorizer.fit_transform(df['text'])
X = X.toarray()
tfidf_df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])

#用tfidf模型将keywords转成词向量的形式,具体参数没研究过.
vectorizer2 = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(1,3),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True,max_features=100,
)
X2 = vectorizer2.fit_transform(df['keywords'])
X2 = X2.toarray()
tfidf_df2 = pd.DataFrame(X2, columns=[f'keywords_{i}' for i in range(X2.shape[1])])

#用tfidf模型将地点转换成词向量表示,具体参数没有研究过.
vectorizer3 = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(1,2),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True
)
X3 = vectorizer3.fit_transform(df['venue'])
X3 = X3.toarray()
tfidf_df3 = pd.DataFrame(X3, columns=[f'venue_{i}' for i in range(X3.shape[1])])

#用普通的词频统计模型将text转成词向量,具体参数没有研究过.
vectorizer_cnt = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(1,3),
            min_df=0.10,
            max_df=0.85,max_features=100
)
train_tfid = vectorizer_cnt.fit_transform([i for i in df['text']])
train_tfid  = train_tfid.toarray()
cnt_df = pd.DataFrame(train_tfid, columns=[f'tfid_cnt_{i}' for i in range(train_tfid.shape[1])])

#用普通的词频统计模型将keywords转成词向量,具体参数没有研究过.
vectorizer_cnt2 = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(2,3),
            min_df=0.10,
            max_df=0.85,max_features=100
)
train_tfid2 = vectorizer_cnt2.fit_transform([i for i in df['keywords']])
train_tfid2 = train_tfid2.toarray()
cnt_df2 = pd.DataFrame(train_tfid2, columns=[f'tfid_cnt2_{i}' for i in range(train_tfid2.shape[1])])

#用普通的词频统计模型将venue转成词向量,具体参数没有研究过.
vectorizer_cnt3 = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(1,2),
            min_df=0.10,
            max_df=0.85,
)
train_tfid3 = vectorizer_cnt3.fit_transform([i for i in df['venue']])
train_tfid3 = train_tfid3.toarray()
cnt_df3 = pd.DataFrame(train_tfid3, columns=[f'tfid_cnt3_{i}' for i in range(train_tfid3.shape[1])])

#用普通的词频统计模型将topauthor转成词向量,具体参数没有研究过.
vectorizer_cnt4 = CountVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(1,2),
            min_df=0.10,
            max_df=0.85,
)
train_tfid4 = vectorizer_cnt4.fit_transform([i for i in df['top_author']])
train_tfid4 = train_tfid4.toarray()
cnt_df4 = pd.DataFrame(train_tfid4, columns=[f'tfid_cnt4_{i}' for i in range(train_tfid4.shape[1])])


#用word2vec将text转换成词向量
processed_texts = [simple_preprocess(text) for text in df['text']]#小写、标点、分词
size = 256#词向量的维度为256维
#训练word2vec模型,传入文本数据processed_texts,词向量维度为256维,上下文窗口为5,即考虑前后2个词,考虑最少出现2次的词,并行训练模型
model = Word2Vec(processed_texts, vector_size=size, window=5, min_count=2, workers=16)
#均值词向量
X = []
for text in processed_texts:
    vector = np.zeros(size)#初始化向量
    count = 0
    for word in text:
        if word in model.wv:#如果一个词在词汇表中
            vector += model.wv[word]#加上对应的向量
            count += 1
    if count > 0:
        vector /= count
    X.append(vector)
w2v = pd.DataFrame(X, columns=[f'w2v_{i}' for i in range(size)])

del vectorizer,vectorizer2,vectorizer3,vectorizer_cnt,vectorizer_cnt2,vectorizer_cnt3,vectorizer_cnt4,model
gc.collect()#手动触发垃圾回收,强制回收由垃圾回收器标记为未使用的内存

df = pd.concat([df.reset_index(drop=True),tfidf_df,tfidf_df2,tfidf_df3,cnt_df,cnt_df2,cnt_df3,cnt_df4,w2v],axis=1)
df.head()

,author_id,name,author,venue,keywords,text,x1,x2,x3,x4,...,w2v_246,w2v_247,w2v_248,w2v_249,w2v_250,w2v_251,w2v_252,w2v_253,w2v_254,w2v_255
0,Iki037dt,224,"[{'name': 'Hideaki Bando', 'org': ''}, {'name'...",2607,the power of the high-sensitive kras test to p...,the power of the high-sensitive kras test to p...,120,0,0,14,...,0.254230,-0.591256,-0.047873,-0.518315,-0.170804,-0.094971,0.009196,-0.261595,0.165669,0.259170
1,Iki037dt,224,"[{'name': 'jintetsu soh', 'org': ''}, {'name':...",18917,1812 nicardipine versus saline injection as tr...,1812 nicardipine versus saline injection as tr...,123,0,0,8,...,0.683561,-0.365802,0.765757,-0.325274,-0.062557,0.341192,0.125490,-0.561785,0.108586,0.246013
2,Iki037dt,224,"[{'name': 'M Inaba', 'org': 'Department of Uro...",19400,"vesicoureteral reflux, bladder hypertrophy, ch...",possible use of ultrasound estimated bladder w...,100,986,4,9,...,0.014613,-0.199763,-0.033882,-0.048321,-0.051605,0.066802,0.160459,-0.193787,0.172351,0.265669
3,Iki037dt,224,"[{'name': 'hisao ito', 'org': ''}, {'name': 'm...",13047,interaction between epidermal growth factor an...,interaction between epidermal growth factor an...,103,0,0,10,...,0.713096,0.010418,0.031974,-0.343218,0.052702,-0.485268,-0.381349,-0.482236,0.081133,0.303045
4,Iki037dt,224,"[{'name': 'yasutoshi kuboki', 'org': 'Departme...",2607,"gastric cancer, amplification, mutation, next-...",comprehensive analyses using next-generation s...,133,1629,5,10,...,0.232979,-0.391160,0.260753,-0.290798,0.068610,0.043475,-0.130058,-0.040746,-0.159678,0.224960


### 6.top100重合度特征

#### 这里先将文本去标点、小写,去掉停用词,统计每个author的top100词,然后统计text中的每个词是top100的占比。比如有个文本分词后是['a','b','c'], 'a'和'b'是top100词库里的,那么占比就是66%.

#### 为什么要去标点、小写、去停用词呢?停用词、标点这种每篇文章都有,无法表现出每个作者各自的特点,如果统计的top100中都是标点、停用词,那可能构造的重合度特征就没什么用了,可能大家都差不多,你一句话里有一个句号,我一句话里也有个句号。

#### 注:我之前自己在比赛中是取出现最频繁的100个词,对title和abstract做词频统计,其中有很多是标点和停用词,所以效果也没有那么好。


In [7]:
#label不是nan的就是训练集,是nan的就是测试集
train,valid = df[~df['label'].isna()].reset_index(drop=True),df[df['label'].isna()].reset_index(drop=True)
del df
gc.collect()#手动触发垃圾回收,强制回收由垃圾回收器标记为未使用的内存
#删除标点和stopwords是为了让后面的top100词带有作者的特点。
def preprocess_text(text):
    #空字符替换成空字符,并删除所有的标点符号
    text = text.translate(str.maketrans("", "", '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'))
    #转换为小写
    text = text.lower()
    #据我分析,他这里可能是把停用词删除掉.
    stopwords=['the','of','a','an','how','what','which','why','where','who','we','are','is','by','base','on','in']
    for s in stopwords:
        text = text.replace(s,'')
    return text

#这里就是统计text里的word是top100word在text里的占比
def calculate_overlap_score(text, top100_words):
    words=text.split()#文本按空格拆分成单词
    #统计words里每个word在top100_words的个数
    overlap_count = sum(int(word in top100_words) for word in words)
    overlap_score = overlap_count / len(words) if len(words) > 0 else 0
    return overlap_score

# 按作者分组
def get_overlap_score(df,col):
    #主要就是文本去除标点符号和停用词
    df[col] = df[col].astype(str).apply(preprocess_text)
    grouped = df.groupby("name")#按照作者的姓名进行分组
    
    #统计每个作者的所有文本中top100words(由于label为0的样本也有标签1,故也可以认为是label=1的样本中top100)
    top100_words = {}
    for author, group in tqdm(grouped):
        #应该是把这个作者的所有文本拼接在一起
        author_label_1_text = " ".join(group[col])
        #分词操作
        words = author_label_1_text.split()
        #返回一个字典,表示每个词出现的词数{'a':2,'b':1}
        word_counts = Counter(words)
        #获取top100的词
        top100 = [word for word, _ in word_counts.most_common(100)]
        #存入字典,author的top100个词就是top100.
        top100_words[author] = top100

    cnt = 0
    for x,y in zip(df['name'],df[col]):#x是作者姓名,y是作者文本
        cnt  += 1
        #文本中有多少词是来自这个作者top100的词
        overlap_score = calculate_overlap_score(y, top100_words[x])
        #这篇论文的分数是多少
        df.loc[cnt-1,f'{col}_overlap_score'] = overlap_score
    return df
#增加文本top100重合度的特征.
for col in ['text','keywords']:
    train = get_overlap_score(train,col)
    valid  = get_overlap_score(valid,col)

100%|██████████| 515/515 [00:00<00:00, 532.58it/s]


### 7.关键特征.

#### 这里特征太多,不一定说的清楚,还是直接看代码吧。

#### 首先是text按照句号、逗号、空格分割统计词数,然后对name,year(也就是x5),‘venue’进行groupby,例如这个人发表的年份有几个(这里是nunique),发表的地点有几个。在这个年份发表的作者有几个,在这个年份有几个地点(或者应该说学术会议)。然后是论文长度和词数的特征对于每个作者是不一样的,每个年份也是不一样的,所以又进行了groupby的统计特征。然后拿一个人的一篇论文和这个人的其他论文进行比较,和同一年发表的其他论文进行比较,这里的比较有作差和tfidf两种特征。上一步统计出来的词向量特征也是和这个人groupby特征进行了一个比较,这里是作差比较。


#### 本次比赛就是要找异常,而异常就是和大多数不一样的数据,所以和groupby的特征作差能够把异常暴露出来。

In [8]:
def stage1_fe(df):

    #统计按照逗号、句号、空格分割字符串的个数
    df["text_content_cnt"] = df['text'].apply(lambda x: len(x.split('.')))
    df["text_sentence_cnt"] = df['text'].apply(lambda x: len(x.split(',')))
    df["text_word_cnt"] = df['text'].apply(lambda x: len(x.split(' ')))

    #x5=2024-year,venue是label_encode的原始地点,id就是数据的index吧,name是作者本人名字
    for col in ['x5','venue','id']:
        #作者的发表年份有几个唯一值,作者发表了几篇论文,作者发表的地点有几个唯一值,作者发表了几篇论文
        #作者idx有几个唯一值,作者idx的count
        df[f'name_{col}_nunique'] = df.groupby(['name'])[col].transform('nunique')
        df[f'name_{col}_count'] = df.groupby(['name'])[col].transform('count')
    for col in ['name','venue','id']:
        #每个年份有几个作者发表,发表了多少篇论文,每个年份有几个地点的论文,发表多少篇论文
        #每个年份有多少个idx,idx的count
        df[f'year_{col}_nunique'] = df.groupby(['x5'])[col].transform('nunique')
        df[f'year_{col}_count'] = df.groupby(['x5'])[col].transform('count')
    #作者写了多少篇论文
    name_dict = df['name'].value_counts().to_dict()
    df['name_count'] = df['name'].map(name_dict)
    #每个地点有多少篇论文
    venue_dict = df['venue'].value_counts().to_dict()
    df['venue_count'] = df['venue'].map(venue_dict)
    
    #论文的各种长度和词数对于每个人的特征统计
    for col in tqdm(['x1', 'x2', 'x3', 'x4', 'x5','x6',
                     "text_content_cnt","text_sentence_cnt","text_word_cnt"]):
        for meth in ['mean','max','min','std','median','skew']:
            df[f'{col}_{meth}_name'] = df.groupby(['name'])[col].transform(meth)
    #论文的各种长度和词数在每个年份的特征    
    for col in tqdm(['x1', 'x2', 'x3', 'x4', 'x6',
                     "text_content_cnt","text_sentence_cnt","text_word_cnt"]):
        for meth in ['mean','max','min','std','median','skew']:
            df[f'year_{col}_{meth}'] = df.groupby(['x5'])[col].transform(meth)

    #词数、句子数
    for col in ["text_content_cnt","text_sentence_cnt","text_word_cnt"]:
        for meth in ['mean','max','min']:
            #和这个人特征的不同
            df[f'{col}_{meth}_name_dif'] = df[f'{col}_{meth}_name'] - df[col]
            df[f'{col}_{meth}_name_ratio'] = df[col]/(1+df[f'{col}_{meth}_name'])
            #和这个年份特征的不同
            df[f'year_{col}_{meth}_dif'] = df[f'year_{col}_{meth}'] - df[col]
            df[f'year_{col}_{meth}_ratio'] = df[col]/(1+df[f'year_{col}_{meth}']) 
    
    #tf-idf,word2vec,count等词频统计的模型和这个人的不同之处
    for col in tqdm(list(tfidf_df.columns)+list(tfidf_df2.columns)+list(tfidf_df3.columns)):
         for meth in ['mean','max','min']:
            df[f'{col}_{meth}_tfidf_dif'] = df.groupby(['name'])[col].transform(meth)-df[col]
    for col in tqdm(list(cnt_df.columns)+list(cnt_df2.columns)+list(cnt_df3.columns)+list(cnt_df4.columns)):
         for meth in ['mean','max','min']:
            df[f'{col}_{meth}_cnt_dif'] = df.groupby(['name'])[col].transform(meth)-df[col]
    for col in tqdm(list(w2v.columns)):
         for meth in ['mean','max','min']:
            df[f'{col}_{meth}_w2v_dif'] = df.groupby(['name'])[col].transform(meth)-df[col]
   
    return df
train = stage1_fe(train)
valid = stage1_fe(valid)
del tfidf_df,tfidf_df2,tfidf_df3,cnt_df,cnt_df2,cnt_df3,w2v
gc.collect()#手动触发垃圾回收,强制回收由垃圾回收器标记为未使用的内存
#author好像从来都没有用过:原始数据中['name','org'],文本特征也基本提取完成了
train = train.drop(['author','text','keywords','venue'],axis=1)
valid = valid.drop(['author','text','keywords','venue'],axis=1)
train.head()

100%|██████████| 256/256 [00:05<00:00, 48.78it/s]


,author_id,name,x1,x2,x3,x4,x5,x6,top_author,label,...,w2v_252_min_w2v_dif,w2v_253_mean_w2v_dif,w2v_253_max_w2v_dif,w2v_253_min_w2v_dif,w2v_254_mean_w2v_dif,w2v_254_max_w2v_dif,w2v_254_min_w2v_dif,w2v_255_mean_w2v_dif,w2v_255_max_w2v_dif,w2v_255_min_w2v_dif
0,Iki037dt,224,120,0,0,14,14,18,"hideaki bando,takayuki yoshino,kazuya tuchihar...",1.0,...,-1.243632,0.212314,1.149465,-0.777461,-0.190124,1.220609,-1.051494,-0.209003,0.730881,-1.148365
1,Iki037dt,224,123,0,0,8,13,22,"jintetsu soh,yoshio naya,atsushi ochiai,yasuyu...",1.0,...,-1.359926,0.512504,1.449654,-0.477271,-0.133041,1.277692,-0.994411,-0.195847,0.744038,-1.135209
2,Iki037dt,224,100,986,4,9,23,32,"m inaba,o ukimura,a kawauchi,t iwata,m kanazawa",1.0,...,-1.394895,0.144506,1.081657,-0.845269,-0.196806,1.213927,-1.058176,-0.215502,0.724383,-1.154864
3,Iki037dt,224,103,0,0,10,37,31,"hisao ito,masanobu yamamoto,naruhito oda,fumio...",1.0,...,-0.853086,0.432955,1.370105,-0.556821,-0.105588,1.305145,-0.966958,-0.252879,0.687006,-1.192241
4,Iki037dt,224,133,1629,5,10,9,18,"yasutoshi kuboki,seigo yamashita,tohru niwa,to...",1.0,...,-1.104377,-0.008535,0.928615,-0.998311,0.135223,1.545956,-0.726147,-0.174793,0.765092,-1.114155


### 8.余弦相似度

#### 就是把词向量特征进行余弦相似度的构造,他这里很聪明,是和上一步均值作差的特征进行余弦相似度计算。如果和论文特征groupby的均值越靠近,那么就是和论文整体是靠近的,那么就应该是normal的数据,而反映到数据上和groupby均值靠近,和均值相减就越接近0,余弦相似度也就是接近0的。另一方面他这样做时间复杂度是O(n)。

#### 我之前尝试余弦相似度的时候是两两数据的词向量计算余弦相似度,这样时间复杂度高,是O(n^2),而且论文的特征和整体相似才是真的相似,和另外一篇论文是否相似其实无关紧要。

In [9]:
#这里就是两组向量余弦相似度的比较(和均值去比较余弦相似度)
def stage2_fe(df,col1,col2,name):
    arr_1 = np.array(df[col1])
    arr_2 = np.array(df[col2])
    cosine_sims = []
    for i in tqdm(range(len(arr_1))):
        vec_1,vec_2 = arr_1[i],arr_2[i]
        cosine_sim = np.dot(vec_1, vec_2) / (np.linalg.norm(vec_1) * np.linalg.norm(vec_2))
        cosine_sims.append(cosine_sim)
    return cosine_sims

w2v_mean_columns = [col for col in train.columns if "mean_w2v_dif" in col]
w2v_columns = [f'w2v_{i}' for i in range(size)]

tfidf2_mean_columns = [col for col in train.columns if "mean_cnt_dif" in col]
cnt_columns = [col for col in train.columns if "tfid_cnt" in col]
all_cnt_columns = cnt_columns[:len(tfidf2_mean_columns)]

w2v_cos_train = stage2_fe(train,w2v_columns,w2v_mean_columns,'w2v')
w2v_cos_valid = stage2_fe(valid,w2v_columns,w2v_mean_columns,'w2v')  

cnt_cos_train = stage2_fe(train,all_cnt_columns,tfidf2_mean_columns,'cnt')
cnt_cos_valid = stage2_fe(valid,all_cnt_columns,tfidf2_mean_columns,'cnt')   

train['w2v_cosine_sims'] = w2v_cos_train
valid['w2v_cosine_sims'] = w2v_cos_valid
train['cnt_cosine_sims'] = cnt_cos_train
valid['cnt_cosine_sims'] = cnt_cos_valid
del w2v_cos_train,w2v_cos_valid,cnt_cos_train,cnt_cos_valid
gc.collect()#手动触发垃圾回收,强制回收由垃圾回收器标记为未使用的内存

100%|██████████| 116262/116262 [00:03<00:00, 35276.04it/s]


0

### 9.reduce_mem_usage

#### 这是我自己加的,因为要在Kaggle上运行,不加的话会超内存。

In [10]:
#遍历表格df的所有列修改数据类型减少内存使用
def reduce_mem_usage(df, float16_as32=True):
    #memory_usage()是df每列的内存使用量,sum是对它们求和, B->KB->MB
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:#遍历每列的列名
        col_type = df[col].dtype#列名的type
        if col_type != object and str(col_type)!='category':#不是object也就是说这里处理的是数值类型的变量
            c_min,c_max = df[col].min(),df[col].max() #求出这列的最大值和最小值
            if str(col_type)[:3] == 'int':#如果是int类型的变量,不管是int8,int16,int32还是int64
                #如果这列的取值范围是在int8的取值范围内,那就对类型进行转换 (-128 到 127)
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                #如果这列的取值范围是在int16的取值范围内,那就对类型进行转换(-32,768 到 32,767)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                #如果这列的取值范围是在int32的取值范围内,那就对类型进行转换(-2,147,483,648到2,147,483,647)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                #如果这列的取值范围是在int64的取值范围内,那就对类型进行转换(-9,223,372,036,854,775,808到9,223,372,036,854,775,807)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:#如果是浮点数类型.
                #如果数值在float16的取值范围内,如果觉得需要更高精度可以考虑float32
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:#如果数据需要更高的精度可以选择float32
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float16)  
                #如果数值在float32的取值范围内，对它进行类型转换
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                #如果数值在float64的取值范围内，对它进行类型转换
                else:
                    df[col] = df[col].astype(np.float64)
    #计算一下结束后的内存
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #相比一开始的内存减少了百分之多少
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
train=reduce_mem_usage(train, float16_as32=False)
valid=reduce_mem_usage(valid, float16_as32=False)

Memory usage of dataframe is 4309.40 MB
Memory usage after optimization is: 966.96 MB
Decreased by 77.6%
Memory usage of dataframe is 3025.59 MB
Memory usage after optimization is: 678.23 MB
Decreased by 77.6%


### 10.Model training


#### 模型训练代码我也做过修改,我想看看用官方评估指标达到的成绩,不过最后由于做过数据增强,线下CV也不准。

In [11]:
def weight_rocauc(authorid,y_true,y_pro):
    unique_authorid=[]
    for a in authorid:
        if a not in unique_authorid:
            unique_authorid.append(a) 
    total_weight_auc=0
    for i in range(len(unique_authorid)):
        cur_idx=np.where(authorid==unique_authorid[i])[0]
        if len(cur_idx)>0:#authorid可能是
            auc=roc_auc_score(y_true[cur_idx],y_pro[cur_idx])
            weight_auc=np.sum(y_true[cur_idx]==0)*auc
            total_weight_auc+=weight_auc
    total_weight_auc/=np.sum(y_true==0)
    return total_weight_auc#'weight_rocauc',total_weight_auc,True#名称,值,是否越高越好

choose_cols=[col for col in valid.drop(['id','label', 'top_author','author_id'],axis=1).columns]
del valid
gc.collect()#手动触发垃圾回收,强制回收由垃圾回收器标记为未使用的内存

def fit_and_predict(train_feats=train,model=None,num_folds=10,seed=2024,name='lgb'):
    X=train_feats[choose_cols].copy()
    y=train_feats['label'].copy()
    authorid=train_feats['author_id'].values
    oof_pred_pro=np.zeros((len(X)))
    del train_feats
    gc.collect()#手动触发垃圾回收,强制回收由垃圾回收器标记为未使用的内存
     
    #10折交叉验证
    sgkf = StratifiedGroupKFold(n_splits=num_folds,shuffle=True)
    for fold, (train_index, valid_index) in (enumerate(sgkf.split(X,y,authorid))):
        print(f"name {name},fold:{fold}")

        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        model.fit(X_train,y_train,eval_set=[(X_valid, y_valid)],
                      callbacks=[log_evaluation(100),early_stopping(100)]
                     )
        
        oof_pred_pro[valid_index]=model.predict_proba(X_valid)[:,1]
        del X_train,X_valid,y_train,y_valid
        gc.collect()#手动触发垃圾回收,强制回收由垃圾回收器标记为未使用的内存
        
    print(f"weight_roc_auc:{weight_rocauc(authorid,y.values,oof_pred_pro)}")
    
    return oof_pred_pro

lgb_params={
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 6,
    "learning_rate": 0.05,
    "n_estimators":10000,
    "colsample_bytree": 0.2,
    "colsample_bynode": 0.2,
    "verbose": -1,
    "random_state": 2024,
    "reg_alpha": 0.1,
    "reg_lambda": 10,
    "extra_trees":True,
    'num_leaves':127,
    "verbose": -1,
    "max_bin":225,
    "class_weight":'balanced',
    }

lgb_oof_pred_pro=fit_and_predict(model=LGBMClassifier(**lgb_params),num_folds=5,seed=2024,name='lgb')

name lgb,fold:0
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.777487
[200]	valid_0's auc: 0.791134
[300]	valid_0's auc: 0.799193
[400]	valid_0's auc: 0.804648
[500]	valid_0's auc: 0.809445
[600]	valid_0's auc: 0.812966
[700]	valid_0's auc: 0.815444
[800]	valid_0's auc: 0.816672
[900]	valid_0's auc: 0.817493
[1000]	valid_0's auc: 0.818861
[1100]	valid_0's auc: 0.820228
[1200]	valid_0's auc: 0.821152
[1300]	valid_0's auc: 0.821613
[1400]	valid_0's auc: 0.822557
[1500]	valid_0's auc: 0.823092
[1600]	valid_0's auc: 0.823445
[1700]	valid_0's auc: 0.823524
[1800]	valid_0's auc: 0.824086
[1900]	valid_0's auc: 0.824145
[2000]	valid_0's auc: 0.824148
Early stopping, best iteration is:
[1931]	valid_0's auc: 0.824365
name lgb,fold:1
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.727946
[200]	valid_0's auc: 0.740052
[300]	valid_0's auc: 0.749021
[400]	valid_0's auc: 0.752548
[500]	valid_0's auc: 0.754369
[600]	valid_0's a

### 我猜测线下CV低的原因可能是作者用了对抗学习,负样本label同时为0和1,由于做了数据增强所以也不知道真实的weight_rocauc.

### 如果有机会我会看看top2的方案.https://github.com/LoveFishoO/2024-KDD-WhoIsWho